# Topic 12 - Word Embeddings

## 1 The trouble with Markov Models
In studying Markov Models, we found that we could produce sequences of words that *very* roughly mimicked the natural patterns of word co-occurence found in the corpus on which the Markov model was trained.  However, a major limitation of this mechanism was that the *only* information that could be used to make a decision about what the next word should be, was the previous word.  This issue is partially ameliorated by higher order Markov Models, namely ones that construct a probability table mapping the last two or three words to a probability of the next word, however this is still a very short distance to look.  Take as an example the following phrase:

**I lived in France for awhile, and during that time I learned to speak ____** 

What would be the order of the Markov model required to correctly fill in the blank?  Given what we know about the number of parameters required for such a model, does this seem feasible?  

What we need instead is a model that has some kind of *selective memory*: it needs to be able to store important contextual information, and be able to reference back to that contextual information later.  In this example, the language model that we produce should be able to recognize the descriptive power of the word France, bundle that information up, carry it along as it progresses through the sequence, and then unpack and conjugate that information when it finds a need.  Some attempts have been made to create a model that performs this task heuristically, through rules-based systems, e.g. when I come a across a noun that occurs in a prepositional phrase, I'm going to save that noun, and when I come across another situation where a pronoun could be used, I'm going to query my saved phrase.  However, most languages have a pretty large and complicated set of syntactical rules, and the way that semantic elements fit into that syntax to produce *meaning* is even more complex.  Obviously, humans don't learn to generate speech that way: instead we build up a kind of empirical model of this structure by sequentially analyzing a large training set (namely all of the things we read and hear, with the added bonus of being able to make targeted queries of that training set to shore up areas we deem to be insufficient).  As such, it makes some sense to imagine that we could train a neural network on this kind of task as well.  Such a neural network would need two components.
- a means of ingesting signals (i.e. words) such that they could be used to make predictions or selections of the next word, in tandem with
- an internal state that could selectively extract, transform, and store the important features from the sequence up until now.
Such an architecture might look something like:
<img src="recurrent_network.png" />
where $x_t$ is the input at time $t$, $h_t$ is the output (perhaps the next word in a sentence, for example), and $A$ is some model with an internal state that is propagated to itself at the next time step.  This type of architectrue is called a *Recurrent Neural Network*, or RNN, and forms the basis of most compelling language simulations.  The specific architecture used to keep track of the model's internal state representing context and meaning turns out to be a little bit finicky, and we'll talk about good choices for the last few sessions of the course.  However, before we do that, we'll need to talk about the first point above: neural networks aren't just probabilistic lookup tables like Markov Models are, but rather they do arithmetic on numbers.  How can we represent words as numbers that neural networks can understand.  The mechanism for doing this ends up not only being useful for creating inputs to RNNs, but also interesting in its own right.

## 2 Words as vectors
A means of turning words into representative vectors turns out to be a sort of byproduct of solving a much simpler problem than language modelling.  Consider the following problem: predict $w_i$ (a word at position $i$), given the words within a given window to the left and to the right.  So for example:  **The orange cat __ down the hill**.  The problem is to teach a computer to fill in the blank.  

Of course you can solve this problem easily, because you've had a lifetime to train your language model, but we'll need a training set to work with.  For instructional purposes, let's use a little bit of Shakespeare.

In [ ]:
corpus = """anarchism originated as a term of abuse first used against early working class 
             radicals including the diggers of the english revolution and the sans culottes 
             of the french revolution whilst the term is still used in a pejorative way to 
             describe any act that used violent means to destroy the organization of society 
             it has also been taken up as a positive label by self defined anarchists the word 
             anarchism is derived from the greek without archons ruler chief king anarchism
             as a political philosophy is the belief that rulers are unnecessary and should 
             be abolished although there are differing interpretations of what this means 
             anarchism also refers to related social movements that advocate the elimination 
             of authoritarian institutions particularly the state the word anarchy as most 
             anarchists use it does not imply chaos nihilism or anomie but rather a harmonious 
             anti authoritarian society in place of what are regarded as authoritarian political 
             structures and coercive economic institutions anarchists advocate social relations 
             based upon voluntary association of autonomous individuals mutual aid and self 
             governance while anarchism is most easily defined by what it is against anarchists also
             offer positive visions of what they believe to be a truly free society however ideas
             about how an anarchist society might work vary considerably especially with respect 
             to economics there is also disagreement about how a free society might be brought 
             about origins and predecessors kropotkin and others argue that before recorded history 
             human society was organized on anarchist principles most anthropologists follow 
             kropotkin and engels in believing that hunter gatherer bands were egalitarian 
             and lacked division of labour accumulated wealth or decreed law and had equal 
             access to resources william godwin anarchists including the the anarchy organisation 
             and rothbard find anarchist attitudes in taoism from ancient china""".split()

We can create a specific training set by looping through the text and extracting a window of text, and using it to predict the central word of the window.

In [ ]:
l = len(corpus)
windowsize = 2
data = []
for i in range(windowsize,l-windowsize):
    target = corpus[i]
    context = corpus[i-windowsize:i] + corpus[i+1:i+windowsize+1]
    data.append([context,target])
for j in range(5):
    print(data[j])

Given the set of four words on the left (the context), we'd like to predict the word on the right.  Perhaps we'd like to do this using a simple multilayer perceptron model.  However, we're still in the same predicament that motivated this exercise: how to encode words as numbers.  **What might be a reasonable way of numerically representing a word?**

We could just assign an integer value to each word: anarchism -> 0, originated -> 1, a -> 2, etc.  However this is problematic because it implies order where none exists: anarchism isn't any "closer" to originated than, say, the word abuse.  Instead, we could use an encoding mechanism we've used before: one-hot encoding.  This makes sense: in a way, we're doing a multiclass classification problem, where the word to predict is the class, and $N$ (the number of classes) is the *vocabulary size*.  The difference now is that we have categorical *features* as well as classes.  Let's just use the same encoding scheme for those features.  For example, with the example above we could have

$$
X_0 = [\underbrace{1}_{\text{anarchism}},\underbrace{1}_{\text{originated}},\underbrace{0}_{\text{as}},\underbrace{1}_{\text{a}},\underbrace{1}_{\text{term}},\underbrace{0}_{\text{of}},\underbrace{0}_{\text{abuse}},\ldots]
$$
$$
Y_0 = [\underbrace{0}_{\text{anarchism}},\underbrace{0}_{\text{originated}},\underbrace{1}_{\text{as}},\underbrace{0}_{\text{a}},\underbrace{0}_{\text{term}},\underbrace{0}_{\text{of}},\underbrace{0}_{\text{abuse}},\ldots]
$$
Note that the one hot matrix *doesn't have any redundancy*: while the above example doesn't show it, if we encountered the word "a" again as we scanned the corpus, that would imply that the 4th element should contain a 1.

## IC12A
Write a function that reads a text corpus and creates a dictionary called word_to_idx that maps from a word to a vocabulary index, and also a complementary dictionary called idx_to_word that maps from a vocabulary index to a word.  There are 180 unique words in the above corpus: your dictionary should thus have 180 entries.

In [ ]:
word_to_idx = {}
idx_to_word = {}
current_index = 0
for word in corpus:
    if word not in word_to_idx.keys():
        word_to_idx[word] = current_index
        idx_to_word[current_index] = word
        current_index += 1
print(word_to_idx)
N = len(word_to_idx.keys())
print(N,l)

Once we have these dictionaries, we can create a dataset that is amenable to use for classification.

In [ ]:
import numpy as np
X = np.zeros((len(data),N))
Y = np.zeros((len(data)))
for i,d in enumerate(data):
    for j in d[0]:
        X[i,word_to_idx[j]] = 1
    Y[i] = word_to_idx[d[1]]

In [ ]:
i_query = 15
print(data[i_query])
print(X[i_query])
print(Y[i_query])

Now we just need to decide on how we want to solve this classification problem.  Let's just use a simple MLP, with a single hidden layer of size $h$.  We can define this in pytorch as usual.

In [ ]:
import torch
import torch.nn

class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.l1 = torch.nn.Linear(vocab_size, embedding_dim)
        self.l2 = torch.nn.Linear(embedding_dim,vocab_size)
      
    def forward(self, x):
        h = self.l1(x)
        out = self.l2(h)
        return out

#    def get_word_emdedding(self, word):
#        word = torch.LongTensor([word_to_ix[word]])
#        return self.embeddings(word).view(1,-1)

In [ ]:
X = torch.tensor(X).to(torch.float)
Y = torch.tensor(Y).to(torch.long)

In [ ]:
import torch.optim
model = CBOW(N,10)
n_epochs = 1000
losses = []
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
for epoch in range(n_epochs):
    optimizer.zero_grad()
        
    # Make a prediction based on the model
    outputs = model(X)
        
    # Compute the loss
    loss = criterion(outputs,Y)      

    # Use backpropagation to compute the derivative of the loss with respect to the parameters
    loss.backward()
        
    # Use the derivative information to update the parameters
    optimizer.step()
    # The loss, the context words, the label, the prediction
    print(loss.item(),data[10][0],data[10][1],idx_to_word[np.argmax(outputs[10].detach().numpy())])    

Okay, so this is neat, now we have a model that takes context words and uses them to fill in the blank.  This isn't all that useful on its own, but think about how many words each row in the weight matrix is actually associated with.  **If I multiply a one-hot encoding of a word by the weight matrix, what do I get out?**
$$
[0,0,1,0,0,\ldots]\begin{bmatrix} W_0 \\ W_1 \\ W_2 \\ W_3 \\ W_4 \\ \vdots \end{bmatrix} = ?
$$
Each row in the weight matrix is associated with just a single word.  Because that matrix row is multiplied by a 1, the resulting functional transformation (i.e. the numbers that come out of the hidden layer for a given word as an input) is simply that row of the matrix (note that these are no activations on this hidden layer).  As such, the rows of the weight matrix can be viewed as a real-valued vector representation of a word.  In some sense, the word has been compressed from its original one-hot representation to a lower-dimensional one, and we can use that representation as the input to other language models.  This transformation is called a *word embedding*.  

For example, we could query the embedding for the word "ruler":

In [ ]:
wv = [i for i in model.l1.parameters()][0].detach().numpy().T
print(wv.shape)
word = 'rulers'
index = word_to_idx[word]
print(wv[word_to_idx[word]])

Thus, to represent the word ruler, instead of the length *N* one-hot encoding, the same information is, in principle, stored in this length *h* vector, as significant space savings, and a significant reduction in dimensionality.  Imagine that you wanted to train another neural network on these features (say, classify sequences of words into having a generally positive or negative connotation, a task called sentiment analysis).  It would take alot less data to constrain a model with 10 features per word than one with $N$ features per word.

## 3 Semantics
However, there's more to the story here.  Dimensionality reduction necessarily involves the loss of information: it's a type of *compression*.  What is the loss of information that's going on here?  Consider the task that we're trying to solve: given context, predict the missing word.  In particular, we're adding up the embeddings of the context words, and using that sum as the set of features used to predict the missing word (via softmax classification),roughly e.g. **belief that rulers are ___ -> unnecessary** uses the sum of the embeddings over belief, that, rulers, and are as the features in a softmax classifier, where unnecessary is the most probable word.  However there are other words that could be substituted in order to retain the same semantic meaning.  For example, we could substitute "kings" for "rulers", and we would hope that we would still get "unnecessary" as a prediction, because the fundamental meaning of the context words when taken together hasn't really changed.  Thus, we would expect the numerical values of the embeddings for "rulers" and "kings" to be very similar to one another.  The idea is to discard the actual nominal word, and to replace it with numbers that encode semantic content: two words that don't sound the same but mean the same thing can be placed very close to one another.  What then should the axes mean?  Well, we don't exactly know, but they are the axes of underlying meaning, and because the meaning of these axes is consistent between all words, we can do some strange and interesting stuff with them.  

Of course we could start by asking directly about semantic similarity: do "kings" and "rulers" actually end up close to one another in this embedding vector space?

In [ ]:
# Compute cosine similarity between rulers and all other words
similarities = (wv[index] * wv).sum(axis=1)/(np.linalg.norm(wv[index])*np.linalg.norm(wv,axis=1))

# Produce a sorted list of similar words from closest to farthest
[idx_to_word[i] for i in np.argsort(similarities)[::-1]]

Disappointingly, it does not.  Why is this?  Well, we're asking the model to learn fairly complex semantics from a passage that is only a few-hundred words long.  This simply isn't enough.  To ameliorate this, we would need to go looking for a much bigger dataset in which each word is used many times in a variety of contexts.  However, this procedure of generating embeddings gets pretty expensive in a hurry: the english vocabulary is vast and having 400k feature one-hot matrices is tough.  However, because of the utility of this idea, there exist a handful of pre-trained models that we can leverage without having to train ourselves.  What does this pre-trained model look like?  It's essentially just a dictionary that maps a word to a vector.  One of the most popular is a model called word2vec, and it's been trained on the googlenews dataset, which is an aggregation of essentially every English language news article ever published, around 3 billion words, with a vocabulary size of approximate 500k.  (Other models have been trained on the entirety of wikipedia, for example).  We can interface with this model using a natural language processing python library called gensim.

In [ ]:
import gensim
model_ = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz',binary=True)

The above line will take a little while to load.  After all it contains a 400k by 300 matrix (these big models use a higher-dimensional embedding vector size).  Once it's loaded, we can ask it some questions.  

In [ ]:
model_.most_similar(positive=['kings'])

"rulers" isn't in there, but we could compute the similarity score ourselves:

In [ ]:
model_.similarity('kings','royalty')

Pretty similar.  Much more similar than, say:

In [ ]:
model_.similarity('kings','catfish')

or

In [ ]:
model_.most_similar(negative=['kings'])

The above command is just saying: negate the vector for 'kings', and search for the vector in the database that is most similar to that negation.  That leaves us with a potentially interesting idea: what happens if we add representations?  For example if we subtract man from king and add woman?
$$
\text{king} - \text{man} + \text{woman} = ?
$$

In [ ]:
model_.most_similar(positive=['king','king'])

It seems that addition might hold over the axes of meaning.  Let's try another one:

In [ ]:
model_.most_similar(positive=['horse','mermaid'],negative=['centaur'])

## IC12B Analogy generator
One thing that word embeddings are good at is analogies (as you can see above).  Write an analogy generator, i.e. write a python function that takes three strings and outputs a fourth such that String 1 is to String 2 as String 3 is to String 4.  What types of analogies can the word embedding model capture?  What does it fail at?  Fun things to try include capital cities to nations (or states), job to place of employment, etc. 